In [1]:
%store -r

In [56]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.concat([df_window0, df_window1, df_window2])
df.window += 1

g = sns.catplot(
    x="size", y="score", data=df, kind="box", hue="method", col="model", row="window"
)
g.set_axis_labels("Number of training samples", "F1 score")
g.set_titles(col_template="Model = {col_name}", row_template="Window size = {row_name}")

plt.savefig("all_results.pdf")

In [52]:
from scipy.stats import ttest_rel

def p(s):
    i = s.keys()[0]
    size = df.iloc[i]["size"]
    method = df.iloc[i]["method"]
    model = df.iloc[i]["model"]

    baseline_scores = df[(df["size"] == size) & (df["model"] == model) & (df["method"] == "Baseline")].score
    scores = s.tolist()
    return ttest_rel(baseline_scores, scores)[1]

In [53]:
for df in [df_window0, df_window1, df_window2]:
    print(f"Window size {df.window.unique()[0] + 1}\n---")
    df = df.drop(columns=["window"])
    print(df.groupby(["size", "model", "method"], sort=False).aggregate([np.mean, np.std, p]))
    
    bl = df[df.method == "Baseline"]["score"]
    gmm = df[df.method == "GMM"]["score"]
    print(f"Baseline mean: {bl.mean()}")
    print(f"GMM mean: {gmm.mean()}")
    print(f"p = {ttest_rel(bl, gmm)[1]}")

Window size 1
---
                           score                    
                            mean       std         p
size model    method                                
200  TokenCNN Baseline  0.831335  0.011848       NaN
              K-Means   0.843846  0.014898  0.031170
              GMM       0.842445  0.010787  0.030924
     CharCNN  Baseline  0.855736  0.011701       NaN
              K-Means   0.851318  0.014746  0.206388
              GMM       0.851584  0.022158  0.586711
800  TokenCNN Baseline  0.842933  0.010159       NaN
              K-Means   0.850113  0.005171  0.045224
              GMM       0.859137  0.014198  0.005521
     CharCNN  Baseline  0.867169  0.018861       NaN
              K-Means   0.863595  0.021780  0.722963
              GMM       0.880255  0.017785  0.141477
1600 TokenCNN Baseline  0.849773  0.006378       NaN
              K-Means   0.859235  0.005771  0.017752
              GMM       0.863006  0.005046  0.000108
     CharCNN  Baseline  0.86